# Trabajo final data mining

En este trabajo brindaremos un chat bot de respuestas cortas para IM en español. Primero que nada definiremos una respuesta corta como cualquier turno hasta 3 palabras que ocurren al menos n veces. Iremos probando diferentes n a modo de prueba. El texto anterior a la respuesta corta será llamado contexto

## Extracción del corpus

Debido a que no hay un corpus bien definido para este problema utilizaremos el contexto del chat como texto de entrenamiento, por lo que la respuesta corta será la etiqueta. Utilizaremos mensajes de whatsapp tomadas de nuestras conversaciones.

### Procesamiento

In [2]:
import re
import csv
import pandas as pd
import glob
import matplotlib.pyplot as plt
import numpy as np

from datetime import datetime, timedelta
from string import punctuation
from collections import defaultdict

In [6]:
files = glob.glob('todos/*')
raw_data = []
for file in files:
    with open(file, 'r') as f:
        raw_data += [f.readlines()]

Sacaremos la primera línea que de nada sirve

`6/18/17, 16:43 - Los mensajes y llamadas en este chat ahora están protegidos con cifrado de extremo a extremo. Toca para más información.`

In [7]:
raw_data[0]

['10/16/17, 17:44 - Messages to this chat and calls are now secured with end-to-end encryption. Tap for more info.\n',
 '10/16/17, 17:44 - \u202a+54 9 351 328-5957\u202c: https://m.facebook.com/story.php?story_fbid=1490068964433492&id=161872773919791\n',
 '10/16/17, 17:54 - Daniela Bosch: Hola Te confundiste de numero\n']

Parte del mismo mensaje :o. Como hacemos? Deberíamos buscar una alternativa

In [90]:
raw_data[18129] + raw_data[18130] + raw_data[18131]

'9/28/17, 21:16 - \u202a+54 9 3544 56-7886\u202c: Pizzas a la parrilla (de esas cocinadas por un chef pulenta), birras y narguilas! \n\nJueves 28, 20.00hs - After Office de Rosh Hashaná! \n'

#### Formato lindo

Para ello spliteo cada línea por la primera ocurrencia de ": ". Ver bien si se puede hacer algo con las líneas que no tengan el mismo formato (por ahora la quitamos). Hay un problema con las fechas debido a la configuración de cada telefono. Algunos formatos es mm/dd/yyyy y otros dd/mm/yyyy. Veremos como solucionarlo en un futuro.

In [18]:
def is_message(text):
    return ': ' in text and '/' in text and ',' in text

def is_continuation(text):
    return '/' not in text or ':' not in text or ',' not in text

def is_beggining(text):
    return 'Messages to this chat and calls are now secured with end-to-end encryption' in text

# at the begining we don't filter any kind of message. Let's see what happend.
info_chat = []
for i, chat in enumerate(raw_data):
    for line in chat:
        line = line.strip('\n')
        if is_beggining(line):
            continue
        if 'location: ' in line:
            continue
        if not is_message(line):
            if is_message(info_chat[len(info_chat)-1][3]):
                if is_continuation(line):
                    # if it is a continuation of a message.
                    info_chat[len(info_chat)-1][3] += '. ' + line
            continue
        info, text = line.split(": ", 1)
        try:
            date, owner = info.split(" - ", 1)
            date, hour = date.split(", ")
        except ValueError:
            if ']' in info:
                date, owner = info.split("] ", 1)
                date, hour = date.split(" ", 1)
                date = date.split("[")[0]
        info_chat.append([i, owner, date, hour, text])

Ahora guardaremos el corpus en formato csv para hacerlo legible. 

Tendrá cinco columnas: numero de chat, persona que escribe el msj, fecha y hora en que se envío, mensaje.

| idchat ||  Owner  ||     Date    || hour ||    Message     |
|--------||---------||-------------||------||----------------|
|    1   || Pedrito || 20-05-2017  || 20:10|| hola que haces |
|    1   ||   Juan  || 21-05-2017  || 12:20|| Todo bien?     |
|    2   ||   Deme  || 15-07-2017  || 13:15||Me fui al baile |

In [19]:
len(info_chat)

186042

In [20]:
info_chat[:10]

[[0,
  '\u202a+54 9 351 328-5957\u202c',
  '10/16/17',
  '17:44',
  'https://m.facebook.com/story.php?story_fbid=1490068964433492&id=161872773919791'],
 [0, 'Daniela Bosch', '10/16/17', '17:54', 'Hola Te confundiste de numero'],
 [1, 'Ori', '6/10/17', '16:58', '😂😂😂😂😂😂😂😂😂😂'],
 [1, 'Luciano', '6/10/17', '17:43', 'Que es este lugar 👀'],
 [1, 'Aida', '6/10/17', '17:44', 'Qur hace este aqui'],
 [1, 'Aida', '6/10/17', '17:44', '\U0001f923'],
 [1, 'Caty', '6/10/17', '17:44', 'Bienvenido'],
 [1,
  'Luciano',
  '6/10/17',
  '17:50',
  'Vengo a pasarle fotos obscenas e irme ?)'],
 [1, 'Luciano', '6/10/17', '17:50', 'Pasarles'],
 [1, 'Caty', '6/10/17', '17:51', '👀']]

In [22]:
fieldnames = ['id', 'owner', 'date', 'hour', 'message']
with open('corpus.csv', 'w', newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames, delimiter=',', quotechar='"', quoting=csv.QUOTE_ALL)
    writer.writeheader()
    
    for idc, owner, date, hour, message in info_chat:
        if message =='':
            continue
        writer.writerow({'id': idc, 'owner': owner, 'date': date, 'hour': hour, 'message': message})

#### Extracción de respuestas cortas

In [80]:
from collections import defaultdict, Counter

short_answers = {}
short_answers = defaultdict(lambda: 0, short_answers)
with open('corpus2.csv', 'r') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        short_answers[row['label']] += 1
        
sorted(short_answers.items(), key=lambda x: x[1], reverse=True)[:10]

[('ja', 111),
 ('si', 16),
 ('', 6),
 ('dale', 6),
 ('eu', 5),
 ('por', 4),
 ('see', 4),
 ('q culia', 3),
 ('noo', 3),
 ('gracias', 2)]

#### Subwords de 3 letras para el contexto

In [57]:
def to_chars(context):
    chars = []
    for line in context:
        line_to_chars = list(line)
        chars += line_to_chars
    return chars

In [58]:
CHARS = 3

def to_subwords(context):
    subwords_ctx = {}
    subwords_ctx = defaultdict(lambda: 0, subwords_ctx)
    for i in range(CHARS-1, len(context)-(CHARS-1)):
        subwords_ctx[context[i-(CHARS-1):i+(CHARS-2)]] += 1
    return subwords_ctx

In [60]:
# TODO: quitar "[", "]" y "'" de los contextos o ver cómo guardarlos
context = {}
context = defaultdict(lambda: {}, context)

with open('corpus2.csv', 'r') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        subwords_ctx = to_subwords(row['context'])
        context[row['label']] = dict(Counter(subwords_ctx) + Counter(context[row['label']]))
print(context['sip'])

{'ert': 1, 'vas': 1, 'owl': 1, 'uen': 2, ' me': 1, ' sh': 1, 'l a': 1, 'r e': 1, 'vis': 1, 'rob': 1, 'ené': 1, 'ati': 1, 'o c': 1, 'ali': 1, 'ry ': 1, 'ue ': 3, 'exa': 1, 'ble': 1, 'ala': 1, 'a y': 2, 's a': 1, 'n t': 1, ' cu': 2, 'ani': 1, 'ués': 1, 'mat': 2, 'da ': 1, 'far': 1, 'she': 1, 'n l': 1, 'ti ': 1, 'pul': 1, 'a n': 2, 'sa ': 1, 'est': 3, 'ori': 1, 'a d': 4, ' en': 2, ' ca': 2, 'ima': 1, 'o m': 1, 'á e': 1, 's e': 5, 'me ': 1, 'mo ': 1, 'ria': 1, 'y q': 1, ' fa': 1, 'ste': 1, 'o l': 1, ' tu': 1, 'tá ': 1, 'esp': 1, 'dad': 2, 'el ': 2, 'sto': 2, 'asa': 1, 'ent': 3, 'uev': 1, ' fo': 1, 'al ': 1, 'sua': 1, 'ese': 2, 'ou ': 1, 'en ': 2, 'tor': 2, 'ta ': 2, 'go ': 1, 'e  ': 1, 'ade': 1, 'nt ': 1, ' la': 3, 'his': 1, 'em ': 1, 'a s': 1, 'cha': 1, ' po': 1, 'e d': 1, 'u p': 1, 'pop': 1, 'asu': 1, 'l m': 1, 'po ': 1, ' an': 1, 'har': 1, ' al': 1, 't j': 1, ' da': 1, ' na': 2, 'e n': 1, 'a m': 1, 'lid': 1, 'o q': 2, 'e h': 1, 'tu ': 1, 'des': 2, 'e b': 1, 'e c': 2, ' bo': 1, 'stá': 2,

### N-Gramas

In [111]:
N = 2

def count_context(sent, n=N):
    c = Counter()
    
    sent_ = ['<s>'] * (n-1) + sent + ['</s>']
    m = len(sent_)
    for i in range(m - n + 1):
        c[tuple(sent_[i:i + n])] += 1
    
    return c

In [112]:
ngram = defaultdict(Counter)
with open('corpus2.csv', 'r') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        sent = row['context'].split(' ')
        ngram[row['label']] += count_context(sent)

ngram['ja']

Counter({('httpswwwfacebook631433487061081', ''): 1,
         ('el', 'jdisnabaixnah'): 1,
         ('de', 'chistes'): 1,
         ('me', 'hizo'): 1,
         ('fue', 'haja'): 1,
         ('digo', 'me'): 1,
         ('mejor', 'que'): 1,
         ('tpb', '</s>'): 1,
         ('a', 'agarrar'): 2,
         ('guada', 'pa'): 1,
         ('me', 'divierte'): 1,
         ('copan', 'uf'): 1,
         ('al', 'menos'): 2,
         ('puntualmente', 'me'): 1,
         ('q', 'ya'): 1,
         ('hacía', 'falta'): 1,
         ('bibliografía', 'bue'): 1,
         ('mano', 'o'): 1,
         ('con', 'guada'): 1,
         ('el', 'capitulo'): 1,
         ('es', 'tpb'): 1,
         ('para', 'mí'): 1,
         ('es', 'domingo'): 1,
         ('globos', 'me'): 1,
         ('juntada', 'hablamos'): 1,
         ('publicacion', 'de'): 1,
         ('una', 'vez'): 2,
         ('lo', 'importante'): 1,
         ('bien', 'y'): 1,
         ('pq', 'nos'): 1,
         ('ponele', 'toda'): 1,
         ('si', '</s>'): 1,
   

In [113]:
%store ngram

Stored 'ngram' (defaultdict)
